In [1]:
import numpy as np
import pandas as pd


In [2]:
df = pd.read_csv('/content/judge-1377884607_tweet_product_company.csv',
                 encoding='unicode_escape')

In [3]:
df.head()

,tweet_text,emotion_in_tweet_is_directed_at,is_there_an_emotion_directed_at_a_brand_or_product
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,iPhone,Negative emotion
1,@jessedee Know about @fludapp ? Awesome iPad/i...,iPad or iPhone App,Positive emotion
2,@swonderlin Can not wait for #iPad 2 also. The...,iPad,Positive emotion
3,@sxsw I hope this year's festival isn't as cra...,iPad or iPhone App,Negative emotion
4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,Google,Positive emotion


In [11]:
df.shape

(9093, 3)

In [4]:
len(df)

9093

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [6]:
# Data preprocessing
X = df[['tweet_text','emotion_in_tweet_is_directed_at']]
y = df['is_there_an_emotion_directed_at_a_brand_or_product']
le = LabelEncoder()
y = le.fit_transform(y)

In [7]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [8]:
# Tokenization and padding
max_words = 1000
max_len = 100
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X)
sequences = tokenizer.texts_to_sequences(X)
X_pad = pad_sequences(sequences, maxlen=max_len)

In [10]:
print("Shape of X_pad:", X_pad.shape)
print("Shape of y:", y.shape)

Shape of X_pad: (2, 100)
Shape of y: (9093,)


In [14]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [15]:
from keras.layers import Dense, LSTM, Embedding
from keras.models import Sequential

In [16]:
model = Sequential()
model.add(Embedding(input_dim=10000, output_dim=5, input_length=500))
model.add(LSTM(50,return_sequences=True))
model.add(LSTM(60))
model.add(Dense(50,activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [17]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 500, 5)            50000     
                                                                 
 lstm (LSTM)                 (None, 500, 50)           11200     
                                                                 
 lstm_1 (LSTM)               (None, 60)                26640     
                                                                 
 dense (Dense)               (None, 50)                3050      
                                                                 
 dense_1 (Dense)             (None, 1)                 51        
                                                                 
Total params: 90941 (355.24 KB)
Trainable params: 90941 (355.24 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [18]:
model.compile(loss="binary_crossentropy", optimizer="adam",
              metrics=["accuracy"])

In [20]:
loss, accuracy = model.evaluate(X_test, y_test)
print("Test Accuracy:", accuracy)